In [ ]:
import os
import json
import pretty_midi
from midi_player import MIDIPlayer
from midi_player.stylers import basic
from corpus.tokenizer import MidiTokenizer

def json_to_midi(notes, output_path):
    midi = pretty_midi.PrettyMIDI()
    instrument = pretty_midi.Instrument(program=0)

    for note in notes:
        instrument.notes.append(
            pretty_midi.Note(
                velocity=note["velocity"],
                pitch=note["pitch"],
                start=note["onset"],
                end=note["offset"],
            )
        )
    midi.instruments.append(instrument)
    midi.write(output_path)

def json_to_midi_with_beat(notes, output_path, beat_json_path):
    midi = pretty_midi.PrettyMIDI()
    
    instrument = pretty_midi.Instrument(program=0)
    for note in notes:
        instrument.notes.append(
            pretty_midi.Note(
                velocity=note["velocity"],
                pitch=note["pitch"],
                start=note["onset"],
                end=note["offset"],
            )
        )
    midi.instruments.append(instrument)
    
    if not os.path.exists(beat_json_path):
        print(f"Missing beat info JSON file: {beat_json_path}")
        return
    with open(beat_json_path, "r") as f:
        beat_info = json.load(f)
    
    metronome_track = pretty_midi.Instrument(
        program=115, is_drum=True, name="Metronome"
    )
    beat_duration = 0.1 
    for downbeat in beat_info.get("downbeat_pred", []):
        note_downbeat = pretty_midi.Note(
            velocity=100,
            pitch=36,
            start=downbeat,
            end=downbeat + beat_duration,
        )
        metronome_track.notes.append(note_downbeat)
    
    for beat in beat_info.get("beat_pred", []):
        note_beat = pretty_midi.Note(
            velocity=100,
            pitch=38,
            start=beat,
            end=beat + beat_duration,
        )
        metronome_track.notes.append(note_beat)
    
    midi.instruments.append(metronome_track)
    
    midi.write(output_path)


def get_midi_player(midi_file_path):
    return MIDIPlayer(url_or_file=midi_file_path, height=600, styler=basic, title='My Player')

base_folder = "../../dataset/0000/17/"
midi_path = base_folder + "aligned_transcription.json"
tempo_path = base_folder + "tempo.json"
beats_path = base_folder + "beats.json"
midi_file_path = base_folder + "detokenize.mid"

tokenizer = MidiTokenizer(tempo_path)
events = tokenizer.encode(midi_path)

# tokenizer.decode_to_score(events)
# restored_notes = tokenizer.restore()

tokenizer = MidiTokenizer(tempo_path)
decoded_notes = tokenizer.decode_to_notes(events)

# json_to_midi_with_beat(notes, midi_file_path, beats_path)
# get_midi_player(midi_file_path)

# json_to_midi(midi_output, midi_file_path)
# get_midi_player(midi_file_path)

# with open(midi_path, 'r') as f:
#     notes = json.load(f)

json_to_midi_with_beat(decoded_notes, midi_file_path, beats_path)
get_midi_player(midi_file_path)

In [ ]:
import json
import pretty_midi
from midi_player import MIDIPlayer
from midi_player.stylers import basic

def json_to_midi(notes, output_path):
    midi = pretty_midi.PrettyMIDI()
    instrument = pretty_midi.Instrument(program=0)

    for note in notes:
        instrument.notes.append(
            pretty_midi.Note(
                velocity=note["velocity"],
                pitch=note["pitch"],
                start=note["onset"],
                end=note["offset"],
            )
        )
    midi.instruments.append(instrument)
    midi.write(output_path)

def get_midi_player(midi_file_path):
    return MIDIPlayer(url_or_file=midi_file_path, height=600, styler=basic, title='My Player')

out_dir = "./infer/output/"
eval_dir = "./evaluation/"
src_dir = "./infer/src/"

extract_path = src_dir + "extract.json"
# origin_path = src_dir + "cover.json"
out_path = out_dir + "output.json"
midi_file_path = out_dir + "picogen.mid"
midi_file_path = out_dir + "output.mid"
# midi_file_path = eval_dir + "JPOP2/etude_e.mid"

with open(extract_path, "r") as f:
    notes = json.load(f)
# with open(origin_path, "r") as f:
#     notes = json.load(f)
with open(out_path, "r") as f:
    notes = json.load(f)


json_to_midi(notes, midi_file_path)


get_midi_player(midi_file_path)

https://youtu.be/6Q0Pd53mojY\?si\=dJuemwnwfdlsgbfk
https://youtu.be/Ug5-kXqP5l8\?si\=WwW9_D6QyBSO6cXZ
https://youtu.be/iFIXi6zzCls?si=kYxheOnqR3573IZp
https://youtu.be/kbNdx0yqbZE?si=4Ze8lkq-LGflsvJE
https://youtu.be/OLRbIc8KZ_8?si=nCbfnyRqRdofOudC 
https://youtu.be/s1bZEnGAX8I\?si\=LigiA3P9sxbBNFwj (musicxml error)
https://youtu.be/4MoRLTAJY_0\?si\=QvLpDCztTiz_wWIT (beat detection error)
https://youtu.be/wgwIfD9Ihik?si=Pap3maz0ho4fv16v
https://youtu.be/JQ2913bVo30?si=7TNGouF9baF_iZWg
https://youtu.be/zjEMFuj23B4?si=kmmHvJ4Wh-ariHIn

#### CPOP

- https://youtu.be/OLRbIc8KZ_8?si=HTPDSGHKPtESid2G 2 3 4
- https://youtu.be/in8NNzwFa-s?si=A9BuyurRE4UPfNtJ
- https://youtu.be/HQ_mU73VhEQ?si=z0Qgj89QVm0P6bVl 4 4 4
- https://youtu.be/8MG--WuNW1Y\?si\=6Y38ZiA2l7ZdKtd9 3 4 4
- https://youtu.be/ZPALMaXLfIw?si=MCc7w7vwrDdlDhNm 3 3 3
- https://youtu.be/h0qYPIlE9us?si=mTG3oz-52Ou2bOi7 2 3 2

---
#### JPOP
https://youtu.be/kbNdx0yqbZE?si=B40NV4X87AqOyx5g 2 2 2
https://youtu.be/Yq7e_AY0dnk?si=KZi6YMpaT6CIWs_g 2 3 4
https://youtu.be/M-Eyhjkepy0?si=wztq11Kp4xCHOqjz 2 3 3
https://youtu.be/fp3F6TqBsAU\?si\=QJ_QvqZoH4HXvf2R 2 2 3
https://youtu.be/bVUEuXOjeDc?si=Wq-ujMPc8qWHnwi3

In [ ]:
from music21 import stream, note, meter, clef, instrument, metadata, layout, duration
import copy

def create_two_handed_piano_score():
    # 建立 Score 物件
    score = stream.Score()
    
    # 加入 metadata（可選）
    score.metadata = metadata.Metadata()
    score.metadata.title = "Two-Handed Piano Score"
    score.metadata.composer = "Composer Name"

    # 建立右手 Part
    right_hand = stream.Part()
    right_hand.id = 'RightHand'
    right_hand.append(instrument.Piano())
    right_hand.append(clef.TrebleClef())
    right_hand.append(meter.TimeSignature('4/4'))

    # 加入右手的音符
    right_hand.append(note.Note("C5", quarterLength=3))
    right_hand.append(note.Note("D5", quarterLength=1))

    # 建立左手 Part
    left_hand = stream.Part()
    left_hand.id = 'LeftHand'
    left_hand.append(instrument.Piano())
    left_hand.append(clef.BassClef())
    left_hand.append(meter.TimeSignature('4/4'))

    # 加入左手的音符
    notes_data = [
        {"pitch": "C4", "onset": 0.0, "duration": 1.0},  # 一般四分音符
        # 八分三連音：1 拍內三個音
        {"pitch": "E4", "onset": 1.0, "duration": 0.333333333, "triplet": True},
        {"pitch": "F4", "onset": 1.333333333, "duration": 0.333333333, "triplet": True},
        {"pitch": "G4", "onset": 1.666666666, "duration": 0.333333333, "triplet": True},
        {"pitch": "G4", "onset": 3.0, "duration": 3.0}
    ]
    t32q = duration.Tuplet(3, 2, 'quarter')

    for ndata in notes_data:
        n = note.Note(ndata["pitch"])

        if ndata.get("triplet"):
            n.duration.type = 'quarter'
            n.duration.appendTuplet(copy.deepcopy(t32q))
        else:
            n.quarterLength = ndata["duration"]

        left_hand.insert(ndata["onset"], n)

    r1 = note.Rest()
    r1.quarterLength = 2.0
    right_hand.insert(4, r1)

    # 將左右手加入 Score
    score.insert(0, right_hand)
    score.insert(0, left_hand)

    # 建立 StaffGroup 並加入 Score
    staff_group = layout.StaffGroup([right_hand, left_hand], name='Piano', symbol='brace')
    score.insert(0, staff_group)

    return score

if __name__ == '__main__':
    score = create_two_handed_piano_score()
    # 或儲存為 MusicXML 檔案
    score.write('musicxml', fp='two_handed_piano_score.musicxml')
